### 1. Kaggle Evaluation Dataset Construction

#### 1.1 Unzip datasets in each folder
- We downloaded datasets using Kaggle's API, resulting in 5,221 folders. Each folder contains a `dataset-metadata.json` file with metadata in the following structure:
  ```
{
  "id":
  "id_no":
  "datasetSlugNullable":
  "ownerUserNullable":
  "usabilityRatingNullable":
  "titleNullable":
  "subtitleNullable":
  "descriptionNullable":
  "datasetId":
  "datasetSlug":
  "hasDatasetSlug":
  "ownerUser":
  "hasOwnerUser":
  "usabilityRating":
  "hasUsabilityRating":
  "totalViews":
  "totalVotes":
  "totalDownloads":
  "title":
  "hasTitle":
  "subtitle":
  "hasSubtitle":
  "description":
  "hasDescription":
  "isPrivate":
  "keywords":
  "licenses":
  "collaborators":
  "data":
}
```
- Each folder also contains a zip file with the corresponding datasets. Our first step is to iterate through all 5,221 folders and unzip all the datasets.

In [ ]:
import os
import zipfile

In [ ]:
# Directory containing the folders with zip files
base_directory = os.path.join(os.getcwd(), "kaggle")

In [ ]:
# # Iterate over all folders in the base directory
# for folder_name in os.listdir(base_directory):
#     folder_path = os.path.join(base_directory, folder_name)
    
#     # Check if it's a directory
#     if os.path.isdir(folder_path):
#         # Look for zip files in the directory
#         for file_name in os.listdir(folder_path):
#             if file_name.endswith(".zip"):
#                 zip_path = os.path.join(folder_path, file_name)
                
#                 # Try to unzip the file
#                 try:
#                     with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#                         zip_ref.extractall(folder_path)
#                     print(f"Unzipped: {zip_path}")
#                 except zipfile.BadZipFile as e:
#                     print(f"Failed to unzip {zip_path}: {e}")

# print("All zip files have been unzipped.")

#### 1.2 Check the files under each folder downloaded from Kaggle

- Original unfiltered # of files in total: 12,533

In [ ]:
import json
import pandas as pd
from tqdm import tqdm

In [ ]:
# Initialize lists
folder_names = []  # list of names of folders containing each dataset
dataset_names = []  # list of "title" fields from dataset-metadata.json
file_names = []  # list of all files in each folder except for dataset-metadata.json and zip files
licenses = []  # list of "licenses name" fields from dataset-metadata.json
descriptions = []  # list of "description" fields from dataset-metadata.json

In [ ]:
# Function to process each Kaggle dataset folder
def process_dataset_folder(folder_path):
    try:
        # Path to the dataset-metadata.json file
        metadata_path = os.path.join(folder_path, 'dataset-metadata.json')
        
        # Read the dataset-metadata.json file
        with open(metadata_path, 'r') as f:
            metadata = json.load(f)

        # Extract the required information
        dataset_name = metadata.get('title', '')
        license_name = metadata.get('licenses', [{}])[0].get('name', '')
        description = metadata.get('description', '')

        # List all files in the folder except for dataset-metadata.json and zip files
        files = [f for f in os.listdir(folder_path) if f != 'dataset-metadata.json' and not f.endswith('.zip')]

        # Store the information in the lists
        for file in files:
            folder_names.append(os.path.basename(folder_path))
            dataset_names.append(dataset_name)
            file_names.append(file)
            licenses.append(license_name)
            descriptions.append(description)
        
    except Exception as e:
        print(f"Error processing folder {folder_path}: {e}")

In [ ]:
# Iterate over each folder in the base directory
for folder_name in tqdm(os.listdir(base_directory), desc="Processing Kaggle Datasets"):
    folder_path = os.path.join(base_directory, folder_name)
    if os.path.isdir(folder_path):
        process_dataset_folder(folder_path)

In [ ]:
# Construct the DataFrame
data = {
    'Folder Name': folder_names,
    'Dataset Name': dataset_names,
    'File Name': file_names,
    'License': licenses,
    'Description': descriptions
}

df = pd.DataFrame(data)

# Adjust display options to show the complete DataFrame
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

In [ ]:
len(df)

#### 1.3 Datasets pre-processing

##### 1.3.1 Filter out csv files
- 12,533 -> 8,629

In [ ]:
# Extract file types (extensions) and make them lowercase
df['File Type'] = df['File Name'].apply(lambda x: os.path.splitext(x)[1].lower())

# Analyze the file types
file_type_counts = df['File Type'].value_counts().reset_index()
file_type_counts.columns = ['File Type', 'Count']

In [ ]:
file_type_counts

In [ ]:
# Filter the original df to include only CSV files
csv_df = df[df['File Type'] == '.csv']

# Reset the index of the filtered DataFrame
csv_df.reset_index(drop=True, inplace=True)

In [ ]:
len(csv_df)

##### 1.3.2 Filter out datasets w/ allowed licenses
- 8,629 -> 7,012

In [ ]:
# Analyze the license
license_counts = csv_df['License'].value_counts().reset_index()
license_counts.columns = ['License', 'Count']

In [ ]:
license_counts

In [ ]:
# Filter the csv_df for allowed licenses
not_allowed_licenses = [
    "unknown",
    "copyright-authors",
]

csv_df_licensed = csv_df[~csv_df['License'].isin(not_allowed_licenses)]

In [ ]:
len(csv_df_licensed)

##### 1.3.3 Filter out datasets w/ description
- 7,012 -> 6,520

In [ ]:
# Filter out datasets with descriptions
csv_df_desc = csv_df_licensed[csv_df_licensed['Description'].str.strip() != ""]

In [ ]:
len(csv_df_desc)

In [ ]:
display(csv_df_desc.head())

##### 1.3.4 Check dataset-table mapping
- 6,520 -> 2,357 (one-to-one mapping only: each dataset contains ONLY one table)

In [ ]:
# Group by Dataset Name and count the number of File Names for each Dataset
table_count_per_dataset = csv_df_desc.groupby('Dataset Name').size().reset_index(name='Table Count')

In [ ]:
table_count_per_dataset

In [ ]:
# Filter datasets that have only one table
one_table_per_dataset = table_count_per_dataset[table_count_per_dataset['Table Count'] == 1]

In [ ]:
len(one_table_per_dataset)

In [ ]:
# Join with the original dataframe `csv_df_desc` to get all other attributes
one_table_datasets = pd.merge(one_table_per_dataset, csv_df_desc, on='Dataset Name')

In [ ]:
display(one_table_datasets.head())

- 6,520 -> 4,163 (one-to-multiple mapping: each dataset contains MULTIPLE tables)

In [ ]:
# Filter datasets that have multiple tables
multi_table_per_dataset = table_count_per_dataset[table_count_per_dataset['Table Count'] > 1]

In [ ]:
multi_table_per_dataset['Table Count'].sum()

In [ ]:
# Join with the original dataframe `csv_df_desc` to get all other attributes
multi_table_datasets = pd.merge(multi_table_per_dataset, csv_df_desc, on='Dataset Name')

In [ ]:
multi_table_datasets

In [ ]:
len(multi_table_datasets)

In [ ]:
# !pip install chardet

import chardet

In [ ]:
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        raw_data = f.read(10000)  # Read only the first 10k bytes
    result = chardet.detect(raw_data)
    return result['encoding']

In [ ]:
def read_csv_with_multiple_encodings(file_path):
    encodings = ['utf-8', 'latin1', 'cp1252']
    detected_encoding = detect_encoding(file_path)
    encodings.insert(0, detected_encoding)  # Try detected encoding first
    for encoding in encodings:
        try:
            return pd.read_csv(file_path, nrows=0, encoding=encoding)  # Read only the header
        except Exception as e:
            continue
    return None  # If all attempts fail

In [ ]:
def check_schema_consistency(folder_name, base_directory):
    folder_path = os.path.join(base_directory, folder_name)
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv') and os.path.isfile(os.path.join(folder_path, f))]
    
    if not csv_files:
        return False

    schemas = []
    for file in csv_files:
        file_path = os.path.join(folder_path, file)
        try:
            df = read_csv_with_multiple_encodings(file_path)
            if df is not None:
                schemas.append(set(df.columns))
            else:
                print(f"Error reading {file_path}: Unable to decode with common encodings")
                return False
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
            return False

    # Check if all schemas are identical
    first_schema = schemas[0]
    for schema in schemas:
        if schema != first_schema:
            return False

    return True

In [ ]:
# Add a column "Schema Consistency" to `multi_table_datasets`
schema_consistency = []

for folder_name in tqdm(multi_table_datasets['Folder Name'].unique(), desc="Checking Schema Consistency"):
    consistency = check_schema_consistency(folder_name, base_directory)
    schema_consistency.append((folder_name, consistency))

In [ ]:
# Convert the schema_consistency list to a DataFrame
schema_consistency_df = pd.DataFrame(schema_consistency, columns=['Folder Name', 'Schema Consistency'])

# Join the schema consistency results with `multi_table_datasets`
multi_table_datasets = pd.merge(multi_table_datasets, schema_consistency_df, on='Folder Name')

In [ ]:
multi_table_datasets['Schema Consistency'].value_counts()

In [ ]:
multi_table_same_schema = multi_table_datasets[multi_table_datasets['Schema Consistency'] == True]
multi_table_diff_schema = multi_table_datasets[multi_table_datasets['Schema Consistency'] == False]

#### 1.4 Visualize # of datasets after different stages of filtering

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Data for the stacked bar chart
categories = ['One-to-One Mapping', 'One-to-Multiple Mapping']
values_one_to_one = [2357, 0]
values_multiple_consistent = [0, 1461]
values_multiple_inconsistent = [0, 2702]

# Data for the funnel chart
stages = [
    "Total Datasets from Kaggle",  # df
    "Filter by CSV",  # csv_df
    "Filter by Licensing",  # csv_df_licensed
    "Filter by Descriptions",  # csv_df_desc
]
counts = [12533, 8629, 7012, 6520]

# Create subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plotting the funnel chart on the first subplot
ax1.barh(stages, counts)

# Adding labels on the bars for the funnel chart
for index, value in enumerate(counts):
    ax1.text(value, index, str(value), va='center')

# Setting title and labels for the first plot
ax1.set_title('Funnel Chart of Dataset Filtering Stages')
ax1.set_xlabel('Number of Files')
ax1.set_ylabel('Filtering Stages')
ax1.invert_yaxis()  # Reverse the order of stages for a funnel effect

# Plotting the stacked bar chart on the second subplot
p1 = ax2.bar(categories, values_one_to_one, label='One-to-One Mapping')
p2 = ax2.bar(categories, values_multiple_consistent, bottom=values_one_to_one, label='One-to-Multiple (Consistent Schema)')
p3 = ax2.bar(categories, values_multiple_inconsistent, bottom=[i+j for i,j in zip(values_one_to_one, values_multiple_consistent)], label='One-to-Multiple (Inconsistent Schema)')

# Add text labels on the bars for the stacked bar chart
def add_labels(bars, ax):
    for bar in bars:
        yval = bar.get_height()
        if yval > 0:
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_y() + yval/2, int(yval), ha='center', va='center', color='white')

add_labels(p1, ax2)
add_labels(p2, ax2)
add_labels(p3, ax2)

# Setting title and labels for the second plot
ax2.set_title('Dataset-Table Mapping')
ax2.set_xlabel('Mapping Type')
ax2.set_ylabel('Number of Tables')
ax2.legend(loc='upper left')

plt.tight_layout()
plt.show()

#### 1.5 Generate keywords, tasks, & other needed metadata for all datasets

In [ ]:
# Save the datasets to CSV files
one_table_datasets.to_csv('one_table_datasets.csv', index=False)
multi_table_same_schema.to_csv('multi_table_same_schema.csv', index=False)
multi_table_diff_schema.to_csv('multi_table_diff_schema.csv', index=False)

In [ ]:
print(f"""
    one_table_datasets header: {one_table_datasets.columns.tolist()}
    multi_table_same_schema header: {multi_table_same_schema.columns.tolist()}
    multi_table_diff_schema header: {multi_table_diff_schema.columns.tolist()}  
""")

In [17]:
import pandas as pd
import os
from tqdm import tqdm
import json

In [7]:
# Load CSV files into dataframe
one_table_datasets = pd.read_csv('raw_kaggle_data/one_table_datasets.csv')
multi_table_same_schema = pd.read_csv('raw_kaggle_data/multi_table_same_schema.csv')
multi_table_diff_schema = pd.read_csv('raw_kaggle_data/multi_table_diff_schema.csv')

In [8]:
from dotenv import load_dotenv
from openai import OpenAI 

In [9]:
# Set the API key and model name
load_dotenv()

MODEL="gpt-4o"
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [10]:
def generate_keywords_and_queries(description, dataset_name, filenames=None):
    if filenames:
        filenames_str = "\n- ".join(filenames)
        
        prompt = f"""
            Given a dataset that contains CSV files with the below file names:
                - {filenames_str}
            And the description of the dataset provided below:
                {description}

            Generate a dictionary in JSON format with the CSV file names as keys and as values a list of 4 semantically distinct data content keywords that describe the expected content of each CSV file but do not describe specific analytic tasks possible with the file.
            For each CSV file, also add a list of 3 semantically distinct analytics task sentences that can be performed with the described CSV file, e.g. develop ML model to predict XYZ.

            Example response format:
            {{
                "csv file 1": {{
                    "keywords": ["keyword1", "keyword2", "keyword3", "keyword4"],
                    "task_queries": ["Task query 1", "Task query 2", "Task query 3"]
                }},
                "csv file 2": {{
                    "keywords": ["keyword1", "keyword2", "keyword3", "keyword4"],
                    "task_queries": ["Task query 1", "Task query 2", "Task query 3"]
                }}
            }}
        """
    else:
        prompt = f"""
            Based on the dataset description provided below, generate a dictionary in JSON format with the dataset name as key and as values a list of 4 semantically distinct data content keywords that describe the expected content of the dataset but do not describe specific analytic tasks possible with the data. 
            Also, add a list of 3 semantically distinct analytics task sentences that can be performed with the described dataset, e.g. develop ML model to predict XYZ.
            If it is hard to complete the task, return an empty dictionary instead.

            Dataset Description:
            "{description}"

            Example response format:
            {{
                "{dataset_name}": {{
                    "keywords": ["keyword1", "keyword2", "keyword3", "keyword4"],
                    "task_queries": ["Task query 1", "Task query 2", "Task query 3"]
                }}
            }}
        """

    response = client.chat.completions.create(
        model=MODEL,
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": "You are a helpful assistant designed generate keywords and task-based queries for tables."},
            {"role": "user", "content": prompt}
        ]
        
    )

    keywords_and_queries = response.choices[0].message.content
    
    try:
        data = json.loads(keywords_and_queries)
        token_usage = response.usage
        return data, token_usage
    except json.JSONDecodeError:
        return {"error": "Failed to decode JSON response"}, response.usage

In [11]:
def process_datasets(one_table_datasets, multi_table_same_schema, multi_table_diff_schema):
    results = []
    token_usages = []

    # Process one_table_datasets and multi_table_same_schema
    for df, table_type in zip([one_table_datasets, multi_table_same_schema], ['one_table', 'multi_table_same_schema']):
        for dataset_name, group in tqdm(df.groupby('Dataset Name'), desc=f"Processing {table_type} datasets"):
            description = group.iloc[0]['Description']
            keywords_and_queries, token_usage = generate_keywords_and_queries(description, dataset_name)
            token_usages.append(token_usage)
            for _, row in group.iterrows():
                file_name = row['File Name']
                keywords = keywords_and_queries.get(dataset_name, {}).get('keywords', [])
                task_queries = keywords_and_queries.get(dataset_name, {}).get('task_queries', [])
                results.append((row['Dataset Name'], file_name, keywords, task_queries, table_type))
    
    # Process multi_table_diff_schema
    for index, row in tqdm(multi_table_diff_schema.iterrows(), desc="Processing multi_table_diff_schema datasets", total=multi_table_diff_schema.shape[0]):
        dataset_name = row['Dataset Name']
        description = row['Description']
        csv_file = row['File Name']
        
        keywords_and_queries, token_usage = generate_keywords_and_queries(description, dataset_name, [csv_file])
        token_usages.append(token_usage)
        keywords = keywords_and_queries.get(csv_file, {}).get('keywords', [])
        task_queries = keywords_and_queries.get(csv_file, {}).get('task_queries', [])
        results.append((dataset_name, csv_file, keywords, task_queries, 'multi_table_diff_schema'))

    return results, token_usages

In [15]:
def create_test_samples(one_table_datasets, multi_table_same_schema, multi_table_diff_schema, n=5):
    one_table_sample = one_table_datasets.sample(n=min(n, len(one_table_datasets)))
    multi_table_same_schema_sample = multi_table_same_schema.sample(n=min(n, len(multi_table_same_schema)))
    multi_table_diff_schema_sample = multi_table_diff_schema.sample(n=min(n, len(multi_table_diff_schema)))
    
    return one_table_sample, multi_table_same_schema_sample, multi_table_diff_schema_sample

In [18]:
# Create 20 test samples
one_table_sample, multi_table_same_schema_sample, multi_table_diff_schema_sample = create_test_samples(one_table_datasets, multi_table_same_schema, multi_table_diff_schema, n=20)

# Process the test samples
test_results, token_usages = process_datasets(one_table_sample, multi_table_same_schema_sample, multi_table_diff_schema_sample)

Processing one_table datasets: 100%|████████████| 20/20 [00:37<00:00,  1.89s/it]
Processing multi_table_same_schema datasets: 100%|█| 15/15 [00:23<00:00,  1.54s/
Processing multi_table_diff_schema datasets: 100%|█| 20/20 [00:53<00:00,  2.69s/


In [19]:
# Print token usage information
for i, usage in enumerate(token_usages):
    print(f"Token usage for request {i+1}: {usage}")

Token usage for request 1: CompletionUsage(completion_tokens=79, prompt_tokens=592, total_tokens=671)
Token usage for request 2: CompletionUsage(completion_tokens=84, prompt_tokens=427, total_tokens=511)
Token usage for request 3: CompletionUsage(completion_tokens=90, prompt_tokens=954, total_tokens=1044)
Token usage for request 4: CompletionUsage(completion_tokens=89, prompt_tokens=430, total_tokens=519)
Token usage for request 5: CompletionUsage(completion_tokens=82, prompt_tokens=326, total_tokens=408)
Token usage for request 6: CompletionUsage(completion_tokens=95, prompt_tokens=533, total_tokens=628)
Token usage for request 7: CompletionUsage(completion_tokens=81, prompt_tokens=307, total_tokens=388)
Token usage for request 8: CompletionUsage(completion_tokens=89, prompt_tokens=441, total_tokens=530)
Token usage for request 9: CompletionUsage(completion_tokens=86, prompt_tokens=370, total_tokens=456)
Token usage for request 10: CompletionUsage(completion_tokens=74, prompt_tokens=4

In [ ]:
# Convert the results to a DataFrame
test_results_df = pd.DataFrame(test_results, columns=['Dataset Name', 'CSV File', 'Keywords', 'Task Queries', 'Table Type'])

In [ ]:
# Join the `test_results_df` back to the corresponding original DataFrames
one_table_sample = pd.merge(one_table_sample, test_results_df[test_results_df['Table Type'] == 'one_table'], 
                            left_on=['Dataset Name', 'File Name'], right_on=['Dataset Name', 'CSV File'], how='left')

multi_table_same_schema_sample = pd.merge(multi_table_same_schema_sample, test_results_df[test_results_df['Table Type'] == 'multi_table_same_schema'], 
                                          left_on=['Dataset Name', 'File Name'], right_on=['Dataset Name', 'CSV File'], how='left')

multi_table_diff_schema_sample = pd.merge(multi_table_diff_schema_sample, test_results_df[test_results_df['Table Type'] == 'multi_table_diff_schema'], 
                                          left_on=['Dataset Name', 'File Name'], right_on=['Dataset Name', 'CSV File'], how='left')

In [ ]:
# Union the DataFrames
union_df_sample = pd.concat([one_table_sample, multi_table_same_schema_sample, multi_table_diff_schema_sample], ignore_index=True)

In [ ]:
union_df_sample

In [ ]:
union_df_sample.to_csv('keyword_query_sample.csv', index=False)